# MSP Simulation Example - Resistive Companion - Exercise 3 - Task 2

## Sample circuit

<img src="Circuit_Exercise_Task2.png" width="400" align="left">

$R_1$ $=1 \Omega$ <br>
$R_2$ $=1 \Omega$ <br>
$C$ $=1mF$ <br>
$L$ $=1mH$ <br>
$I$ $=10A$ <br> 
$v_C(0)$=0 <br>
$i_L(0)=0 $  <br>

## Circuit and Simulation Setup

In [1]:
import numpy as np
import ipywidgets as widget
import matplotlib.pyplot as plt
np.set_printoptions(sign=' ')

# Circuit parameters
R1 = 1
R2 = 1
C1 = 1e-3
L1 = 1e-3
I_src= 10
G1 = 1/R1
G2 = 1/R2

## DC equivalent of Inductor

### Equation of the inductor:

$v_L(t) =L\cdot\frac{di_L}{dt}$

### Integration:   

$i_L(t+\Delta{t}) =i_L(t) + \frac{1}{L} \int_{t}^{t+\Delta{t}} \ v_L(\tau) d\tau$

### Discretization:   

$i_L(k+1)= i_L(k) + \frac{1}{L} \int_{t_k}^{t_k+\Delta{t}} \ v_L(\tau) d\tau$

### Application of trapezoidal rule:  

$i_L(k)= i_L(k) + \frac{1}{L} \frac{v_L(k)+v_L(k+1)}{2} \Delta{t} $

$i_L(k+1)=\frac{\Delta{t}}{2L} v_L(k+1) + (i_L(k) + \frac{\Delta{t}}{2L} v_L(k))  $

The inductor in the calculation step (k + 1) can be substituted with an inductance  $G_L = \frac{\Delta{t}}{2L}$ in parallel with a current source $A_L(k) = i_L(k) + \frac{\Delta{t}}{2L} v_L(k) $ 

<img src="Circuit_Exercise_Task2_DCequivalents.png" width="400" align="left">

## Initialize circuit

In [2]:
# Initialize simulation with DPsim
import dpsimpy

model_name_pf = 'CS_R1R2C1L1_PF'
time_step = 1e-3
final_time = 2e-2

# Nodes
gnd_pf = dpsimpy.sp.SimNode.gnd
n1_pf = dpsimpy.sp.SimNode('n1_pf', dpsimpy.PhaseType.Single)

# Components
r1_pf = dpsimpy.sp.ph1.Resistor('r1_pf', dpsimpy.LogLevel.debug)
r1_pf.set_parameters(1)
r2_pf = dpsimpy.sp.ph1.Resistor('r2_pf', dpsimpy.LogLevel.debug)
r2_pf.set_parameters(1)

cs_pf = dpsimpy.sp.ph1.CurrentSource('cs_pf', dpsimpy.LogLevel.debug)
cs_pf.I_ref = complex(10,0)
cs_pf.f_src = 0

cs_pf.connect([gnd_pf, n1_pf])
r1_pf.connect([n1_pf, gnd_pf])
r2_pf.connect([n1_pf, gnd_pf])

system_pf = dpsimpy.SystemTopology(50, [gnd_pf, n1_pf], [cs_pf, r1_pf, r2_pf])

logger_pf = dpsimpy.Logger(model_name_pf)
logger_pf.log_attribute('n1.v', 'v', n1_pf)
logger_pf.log_attribute('i_r1', 'i_intf', r1_pf)
logger_pf.log_attribute('i_r2', 'i_intf', r2_pf)

sim_pf = dpsimpy.Simulation(model_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.set_time_step(time_step)
sim_pf.set_final_time(final_time)
sim_pf.add_logger(logger_pf)
sim_pf.run()

AttributeError: module 'dpsimpy.sp.ph1' has no attribute 'CurrentSource'

## Run Simulation

In [ ]:
# Simulation parameters
model_name = 'CS_R1R2C1L1'
npoint = int(np.round(final_time/time_step))

# Nodes
gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode('n1')
n1.set_initial_voltage(complex(5,0), 0)
n2 = dpsimpy.emt.SimNode('n2')
n2.set_initial_voltage(complex(5,0), 0)

# Components
r1 = dpsimpy.emt.ph1.Resistor('r1', dpsimpy.LogLevel.debug)
r1.R = 1
r2 = dpsimpy.emt.ph1.Resistor('r2', dpsimpy.LogLevel.debug)
r2.R = 1

c1 = dpsimpy.emt.ph1.Capacitor('c1', dpsimpy.LogLevel.debug)
c1.C = 1e-3
l1 = dpsimpy.emt.ph1.Inductor('l1', dpsimpy.LogLevel.debug)
l1.L = 1e-3
cs = dpsimpy.emt.ph1.CurrentSource('cs', dpsimpy.LogLevel.debug)
cs.I_ref = complex(10,0)
cs.f_src = 0

cs.connect([gnd, n1])
r1.connect([n1, gnd])
c1.connect([n1, n2])
l1.connect([n2, gnd])
r2.connect([n2, gnd])


system = dpsimpy.SystemTopology(50, [gnd, n1, n2], [cs, r1, r2, c1, l1])

logger = dpsimpy.Logger(model_name)
logger.log_attribute('n1.v', 'v', n1)
logger.log_attribute('n2.v', 'v', n2)
logger.log_attribute('3_i_r1', 'i_intf', r1)
logger.log_attribute('4_v_c1', 'v_intf', c1)
logger.log_attribute('5_i_l1', 'i_intf', l1)
logger.log_attribute('6_i_r2', 'i_intf', r2)

# sim = dpsimpy.Simulation(model_name, system, timestep=time_step, duration=final_time, pbar=True, sim_type=1, log_level=1)
sim = dpsimpy.Simulation(model_name)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.do_init_from_nodes_and_terminals(False)

# initialize currents and voltages
print(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0][0])
r1.set_intf_current([[complex(5,0)]])
r1.set_intf_voltage([[5]])
r2.set_intf_current([[5]])
r2.set_intf_voltage([[5]])
c1.set_intf_current([[5]])
c1.set_intf_voltage([[0]])
l1.set_intf_current([[0]])
l1.set_intf_voltage([[5]])

sim.run()

## Read log

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import re

work_dir = 'logs/'
log_path = work_dir + model_name + '.csv'
# log_lines, log_sections = read_dpsim_log(log_path)


## Read solution log

In [ ]:
log_path = work_dir + model_name + '.csv'
ts_dpsim_emt = read_timeseries_dpsim(log_path, print_status=False)

# for key, val in ts_dpsim_emt.items():
    # print(key + ': ' + str(val.values))

## Plot results

In [ ]:
import matplotlib.pyplot as plt

#Extract plot data
plot_data_1=ts_dpsim_emt["n1.v"]
y_values_1= np.asarray(plot_data_1.values)

plot_data_2=ts_dpsim_emt["n2.v"]
y_values_2= np.asarray(plot_data_2.values)

t = np.arange(npoint)*time_step

plt.figure(figsize=(10,8))
plt.xlabel('Time [s]')
plt.ylabel('Voltage [V]')
plt.axis([0, final_time+time_step , -2, 11])
plt.scatter(t,y_values_1, label='$e_{1}$(t)')
plt.scatter(t,y_values_2, label='$e_{2}$(t)')

#show corresponding values on the plot
for i in np.arange(3):
    plt.annotate('  ' + str(np.round(y_values_1[i], 2)), (t[i], y_values_1[i]))
    plt.annotate('  ' + str(np.round(y_values_2[i], 2)), (t[i], y_values_2[i]))
plt.grid()
plt.legend(loc='upper right')
plt.show()